In [7]:
import requests,datetime,io
import pandas as pd
import numpy as np
from scipy import optimize
from requests import Session

SBI = True
TIER = 1

In [8]:
# Read the Transactions File

tdf = pd.read_csv("npsTransactions.csv")
def negUnits(x):
    if(x[0] == "("):
        x = -float(x[1:-1])
    else:
        x = float(x)
    return x
tdf['Amt'] = tdf['Amt'].apply(negUnits)

tdf['Date'] = pd.to_datetime(tdf['Date'])
tdf = tdf.sort_values(by='Date',ascending=True, ignore_index=True)
tdf['Date'] = tdf['Date'].dt.strftime('%Y-%m-%d')

In [9]:
# Initialization and functions
def getNifty(date):
	url="https://archives.nseindia.com/content/indices/ind_close_all_%s.csv"%date
	s=requests.get(url,timeout=5).content
	indicesDf=pd.read_csv(io.StringIO(s.decode('utf-8')))
	indicesDf = indicesDf.loc[indicesDf['Index Name'] == 'Nifty 50']
	niftyClose = indicesDf.at[0,'Closing Index Value']
	return float(niftyClose/100)

def sbiNAV(tier = 1):
	s = Session()
	h = {
		"Host": "www.sbipensionfunds.com",
		"X-Requested-With": "XMLHttpRequest",
		"pragma": "no-cache",
		"sec-fetch-dest": "empty",
		"sec-fetch-mode": "cors",
		"sec-fetch-site": "same-origin",
		"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36",
		"Accept": "*/*",
		"Accept-Encoding": "gzip, deflate, br",
		"Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
		"Cache-Control": "no-cache",
		"Connection": "keep-alive",
		}
	s.headers.update(h)
	req = s.get("https://www.sbipensionfunds.com/latest-nav-2/")
	hdfc = pd.read_html(req.text)[0]
	nav = hdfc.iloc[1,3*tier]
	date = datetime.datetime.strptime(hdfc.iloc[1,0], "%Y-%m-%d")
	return float(nav),date

def hdfcNAV(tier = 1):
	req = requests.get("https://www.hdfcpension.com/nav/")
	hdfc = pd.read_html(req.text)[0]
	nav = hdfc.iloc[6+tier*2,1]
	date = datetime.datetime.strptime(hdfc.columns[1].split("on")[1][1:], "%d-%m-%Y")
	return nav,date

# Functions
def xnpv(rate,cashflows):
    chron_order = sorted(cashflows, key = lambda x: x[0])
    t0 = chron_order[0][0]
    return sum([float(cf)/(1+rate)**((t-t0).days/365.0) for (t,cf) in chron_order])

def xirr(cashflows,currDate,current,guess=0.1):
	for i in range(len(cashflows)):
		t = np.array(cashflows[i][0].split('-'),dtype=int)
		cashflows[i][0] = datetime.datetime.strptime(cashflows[i][0], "%Y-%m-%d") 
	cashflows.append((currDate,-current))
	return optimize.newton(lambda r: xnpv(r,cashflows),guess)

In [10]:
# Units alloted if invested in SBI vs HDFC vs Nifty
df = tdf.copy()
df["Nifty NAV"] = np.zeros(len(df['Date']))
df["NPS Units"] = df["Amt"]/df["NAV"]
df["Nifty Units"] = np.zeros(len(df['Date']))

for i in range(len(df['Date'])):
    inv = float(df.at[i,'Amt'])
    date = datetime.datetime.strptime(df.at[i,'Date'], "%Y-%m-%d")
    try:
        df.at[i,"Nifty NAV"] = getNifty(date.strftime("%d%m%Y"))
        df.at[i,"Nifty Units"] = inv/df.at[i,"Nifty NAV"]
    except:
        date = date - datetime.timedelta(days=1)
        df.at[i,"Nifty NAV"] = getNifty(date.strftime("%d%m%Y"))
        df.at[i,"Nifty Units"] = inv/df.at[i,"Nifty NAV"]
    #print(i,df.at[i,"Nifty NAV"])

In [11]:
# Compute Present Value
df['NPS Units'] = df['NPS Units'].apply(lambda x: round(x, 4))
df['Nifty Units'] = df['Nifty Units'].apply(lambda x: round(x, 4))

invested = df['Amt'].sum()
hdfcUnits = df['NPS Units'].sum()
niftyUnits = df['Nifty Units'].sum()

if SBI:
    nav,date = sbiNAV(tier = TIER)
else:
    nav,date = hdfcNAV(tier = TIER)

nifNAV = getNifty(date.strftime("%d%m%Y"))

hdfcCurr = hdfcUnits*float(nav)
niftyCurr = niftyUnits*nifNAV

xirrHDFC = 100*xirr(((df[["Date", "Amt"]]).values.tolist()),date,hdfcCurr)
xirrNIFTY = 100*xirr(((df[["Date", "Amt"]]).values.tolist()),date,niftyCurr)

index = len(df['Date'])
df.at[index,"Date"] = date.strftime("%d-%b-%Y")
df.at[index,"Amt"] = invested
df.at[index,"NAV"] = nav
df.at[index,"Nifty NAV"] = nifNAV
df.at[index,"NPS Units"] = round(hdfcCurr,2)
df.at[index,"Nifty Units"] = round(niftyCurr,2)
df.at[index+1,"NPS Units"] = str(round(xirrHDFC,2))+"%"
df.at[index+1,"Nifty Units"] = str(round(xirrNIFTY,2))+"%"

In [12]:
# Save to file
df.to_csv("NPS_XIRR.csv",index=False)